In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import math
import numpy as np
import datetime
%matplotlib inline
plt.rcParams['font.family'] = ['SimHei'] #  set font for chinese in matplotlib figure
pd.set_option('display.max_columns', None)

In [ ]:
df_policy_raw = pd.read_csv('../Tbrain_Insurance/policy_claim/policy_0702.csv')
df_claim_raw = pd.read_csv('../Tbrain_Insurance/policy_claim/claim_0702.csv')
df_training_raw = pd.read_csv('../Tbrain_Insurance/training-set.csv')
df_testing_raw = pd.read_csv('../Tbrain_Insurance/testing-set.csv')

In [ ]:
df_p = df_policy_raw.copy() 
df_c = df_claim_raw.copy() 
df_train = df_training_raw.copy() 
df_test = df_testing_raw.copy() 

# Policy

In [ ]:
# Policy_Number	保單號碼 
# Insured's_ID	被保險人代號(替代值) 
# Prior_Policy_Number	前保單號     看 Cancellation & drop 掉
# Cancellation	非繼承自前保單  補成 0 & 1
# Vehicle_identifier	車牌號碼               # 算完ＩＤ擁有車輛數目後drop掉
# Vehicle_Make_and_Model1	廠牌名稱一  # drop 
# Vehicle_Make_and_Model2	廠牌名稱二  # drop
# Manafactured_Year_and_Month	製造年份  # 轉車齡
# Engine_Displacement_(Cubic_Centimeter)	排氣量 # drop 看重置成本
# Imported_or_Domestic_Car	車系代號 # dummy
# Coding_of_Vehicle_Branding_&_Type	廠牌車型代號 #drop
# qpt	乘載數量 # 保留值
# fpt	乘載單位  #drop
# Main_Insurance_Coverage_Group	險種分類  #dummy
# Insurance_Coverage	險種代碼 # dummy
# Insured_Amount1	保額一  # keep
# Insured_Amount2	保額二  # keep
# Insured_Amount3	保額三 # keep
# Coverage_Deductible_if_applied	自負額  # 展開成 (險種abc) x (比例, 數字) + 是否有負號, 注意percentage數值 注意是否與自付額無關的險種
# Premium	簽單保費 # 分三險種加總 + total 4欄,
# Replacement_cost_of_insured_vehicle	重置成本
# Distribution_Channel	主通路代號  # 用Distribution_Channel跑單率替代
# Multiple_Products_with_TmNewa_(Yes_or_No?)	同一車主前一年度非車險保單件數 # keep
# lia_class	車責本期級數  # keep
# plia_acc	車責賠款係數  # +1 keep
# pdmg_acc	車體賠款係數  # +1 keep
# fassured	被保險人性質 # dummy
# ibirth	出生日期  # 轉年齡 自然人ＫＮＮ補 自然人drop
# fsex	性別  # 法人model drop
# fmarriage	婚姻狀況 # 同上
# aassured_zip	郵遞區號 # drop
# iply_area	承保地區代號(分公司)  #dummy
# dbirth	要保人出生日期  # 同出生日期
# fequipment1	配備-音響註記  #dummy
# fequipment2	配備-車箱註記  #dummy
# fequipment3	配備-冷凍機組註記  #dummy
# fequipment4	配備-車框、車斗註記  # drop
# fequipment5	配備-昇降系統註記  #dummy
# fequipment6	配備-電動車電池註記 # drop
# fequipment9	配備-其他  #dummy
# nequipment9	配備-其他說明 #dummy blank->.0  not blank -> 1
##################################### 生feature
# Insured's_ID有幾輛車
# Insured's_ID有張保單
# Insured's_ID 車責 竊盜 車損 單數的 sum, mean, std
# Main_Insurance_Coverage_Group count
# Insurance_Coverage count
# sum, std, mean
# Insured_Amount1_車責 Insured_Amount1_竊盜 Insured_Amount1_車損
# Insured_Amount2 同上
# Insured_Amount3 同上
# (Insured_Amount1 + Insured_Amount2 + Insured_Amount3) * 3 (Insurance_Coverage)
# 3險種  total 保額  / total 保費
# total保費 / 重置成本
# Distribution_Channel 跑單率
# lia_class 係數轉換表 (x-4) * 0.1 + 1
# fequipment1~fequipment9, nequipment9 sum != 0
# ibirth != dbirth

In [ ]:
# df_policy.iloc[:1000,:].groupby(['Policy_Number', 'Prior_Policy_Number'])['Insured\'s_ID'].unique()
df_policy.iloc[:1000,:].groupby(['Policy_Number', 'Prior_Policy_Number', 'Insured\'s_ID']).Vehicle_identifier.nunique().value_counts()

In [ ]:
df_policy[df_policy.Policy_Number == '04ad40d5b2d9e743efaf5ae062f4f7a44b31982b']

In [ ]:
# Policy NAN column
# Prior_Policy_Number    328557  相當於cancellation（Y,N） , drop
# Vehicle_identifier           513        #  drop: Insured ID算車牌個數 
# ibirth                              317765  # 自然人KNN補  法人不補 分model補  optional: age group
# dbirth                            258932   # 自然人KNN補  法人補成NAN 分model補  optional: age group
# fsex                               41945 # 法人補成NAN
# fmarriage                      41945 # 法人補成NAN


In [ ]:
cond = (df_policy.fassured == 2) | (df_policy.fassured == 4)
df_fa = df_policy[cond]
df_policy[cond]['fmarriage'].value_counts(), df_policy[cond]['fmarriage'].isnull().sum()

In [ ]:
df_policy[cond]['dbirth'].isnull().value_counts()

In [ ]:
df_fa[pd.to_datetime(df_fa.dbirth, format='%m/%Y', errors='coerce').isnull() == False]
df_fa_not_null = df_fa[pd.to_datetime(df_fa.dbirth, format='%m/%Y', errors='coerce').isnull() == False]
plt.figure(figsize=(15,5))
(2016 - pd.to_datetime(df_fa_not_null.dbirth, format='%m/%Y', errors='coerce').dt.year).value_counts().sort_index().plot.bar()

In [ ]:
df_fa_not_null[['Premium', 'dbirth']]
df_policy['dbirth'] = (2016 - pd.to_datetime(df_fa_not_null.dbirth, format='%m/%Y', errors='coerce').dt.year)
# df_policy[df_policy.dbirth.isnull() == False][['Premium', 'dbirth']]
df_policy[['Premium', 'dbirth']]

In [ ]:
df_p = df_p.drop(columns=['Prior_Policy_Number', 'Vehicle_Make_and_Model1', 'Vehicle_Make_and_Model2', 
                          'Engine_Displacement_(Cubic_Centimeter)', 'Coding_of_Vehicle_Branding_&_Type', 'fpt', 'aassured_zip', 'fequipment4', 'fequipment6'])

In [ ]:
df_p.Cancellation = df_p.Cancellation.replace(' ', 'N')

In [ ]:
# get dummy
df_p = pd.get_dummies(df_p, drop_first=True, columns=['Cancellation', 
                                                      'Imported_or_Domestic_Car', 
                                                      'Main_Insurance_Coverage_Group', 
                                                      'Insurance_Coverage', 
                                                      'fassured', 
                                                      'fsex', 
                                                      'fmarriage', 'iply_area',
                                                     'fequipment1', 'fequipment2', 'fequipment3', 'fequipment5', 'fequipment9'])

In [ ]:
df_p['Premium_車責'], df_p['Premium_車損'], df_p['Premium_竊盜'], df_p['Premium_total'] = 0, 0, 0, df_p['Premium']


In [ ]:
df_p.loc[df_p.Main_Insurance_Coverage_Group == '車責', 'Premium_車責'] = df_p['Premium']
df_p.loc[df_p.Main_Insurance_Coverage_Group == '車損', 'Premium_車損'] = df_p['Premium']
df_p.loc[df_p.Main_Insurance_Coverage_Group == '竊盜', 'Premium_竊盜'] = df_p['Premium']

In [ ]:
# 暫時不用營業處 (有些testing data 營業處不能算烙跑率)
# distribution channel nunique:   total: train: test = 875
# # distribution channel v.s. prioir policy number / next premium = 0 ( 看有沒有很爛的營業處）
# df_temp = pd.DataFrame(columns=['is0', 'total'])

# distribution_cannels = df_p.Distribution_Channel.unique()
# for distribution_cannel in distribution_cannels:
#     policy_no = df_p[df_p.Distribution_Channel ==distribution_cannel].Policy_Number.unique()
#     policy_count = len(policy_no)
#     next_premium_0_count = df_train[(df_train['Policy_Number'].isin(policy_no)) & (df_train.Next_Premium == 0)].Policy_Number.count()
#     df_temp = df_temp.append(pd.Series({'is0': next_premium_0_count, 'total': policy_count}, name=distribution_cannel))
# df_temp['escape_proportion'] = df_temp['is0'] / df_temp.total
# df_p.Distribution_Channel
# df_p['distribution_channel_escape_proportion'] = df_temp.loc[df_p.Distribution_Channel].escape_proportion.values

In [ ]:
df_p.plia_acc = df_p.plia_acc + 1

In [ ]:
df_p.pdmg_acc = df_p.pdmg_acc + 1

In [ ]:
def check_space(x):
    if x.isspace():
        return 0
    else:
        return 1
df_p.nequipment9 = df_p.nequipment9.map(check_space)

In [ ]:
# 保人擁有車輛數
df_p['InsuredID_car_count'] =df_p['Insured\'s_ID'].map(df_p.groupby('Insured\'s_ID').Vehicle_identifier.nunique())

In [ ]:
# 保人保單數
df_p['InsuredID_policy_count'] = df_p['Insured\'s_ID'].map(df_p.groupby('Insured\'s_ID').Policy_Number.nunique())